In [1]:
import pandas as pd
import numpy as np

#CNN
import keras.layers.core as core
import keras.layers.convolutional as conv
import keras.models as models
import keras.utils.np_utils as kutils

#randomforest
from sklearn.ensemble import RandomForestClassifier

#ㅜnaive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from datetime import datetime

import csv
import gzip
import shutil

Using Theano backend.


In [2]:
#todo check blank value

In [ ]:
#GauusianNB -> 9.208

### preprocess train data

In [2]:
train = pd.read_csv("./train.csv")

In [3]:
def cleanDate(df):
    #clean up Dates
    #split Dates into date and time
    print(df['Dates'][0].split())

    epoch = datetime.utcfromtimestamp(0)
    f = lambda x: (datetime.strptime(x.split()[0], '%Y-%m-%d') - epoch).total_seconds()
    df['date'] = df['Dates'].map(f)
    df['date'] = df['date'] / df['date'].max()
    print(len(df['date'].unique()))

    def getSeconds(x):
        d = datetime.strptime(x.split()[1], '%H:%M:%S')
        return d.hour * 3600 + d.minute * 60 + d.second

    df['time'] = df['Dates'].map(getSeconds)
    print(len(df['time'].unique()))

    df = df.drop('Dates', axis=1)
    return df

In [4]:
def addDummiesAndDrop(df, name):
    #clean up category
    print(df[name].unique())
    nameC = pd.get_dummies(df[name])
    df = df.drop(name, axis=1)
    df = df.join(nameC)
    return df

In [5]:
categoryC = sorted(train['Category'].unique().tolist())
districtC = sorted(train['PdDistrict'].unique().tolist())

In [6]:
train = cleanDate(train)
# train = addDummiesAndDrop(train, 'Category')
train = addDummiesAndDrop(train, 'DayOfWeek')
train = addDummiesAndDrop(train, 'PdDistrict')

['2015-05-13', '23:53:00']
2249
1439
['Wednesday' 'Tuesday' 'Monday' 'Sunday' 'Saturday' 'Friday' 'Thursday']
['NORTHERN' 'PARK' 'INGLESIDE' 'BAYVIEW' 'RICHMOND' 'CENTRAL' 'TARAVAL'
 'TENDERLOIN' 'MISSION' 'SOUTHERN']


In [7]:
#drop descript
train = train.drop('Descript', axis=1)

#drop resolution (not in test.csv)
train = train.drop('Resolution', axis=1)

#drop address
train = train.drop('Address', axis=1)

In [8]:
train.head(1)

,Category,X,Y,date,time,Friday,Monday,Saturday,Sunday,Thursday,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,WARRANTS,-122.425892,37.774599,1.0,85980,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#change category from string to unique int
labels , levels = pd.factorize(train['Category'])
train['Category'] = labels

In [10]:
train.head(1)

,Category,X,Y,date,time,Friday,Monday,Saturday,Sunday,Thursday,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,-122.425892,37.774599,1.0,85980,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [36]:
#not used
#get Category to front
tc = train.columns.tolist()
for i in range(0,4):
    tc.append(tc.pop(0))
train = train[tc]

In [16]:
# train.head(1)

### preprocess test data

In [11]:
test = pd.read_csv("./test.csv")

In [12]:
test.head(1)

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051


In [13]:
test = cleanDate(test)
test = addDummiesAndDrop(test, 'DayOfWeek')
test = addDummiesAndDrop(test, 'PdDistrict')

['2015-05-10', '23:59:00']
2266
1439
['Sunday' 'Saturday' 'Friday' 'Thursday' 'Wednesday' 'Tuesday' 'Monday']
['BAYVIEW' 'NORTHERN' 'INGLESIDE' 'TARAVAL' 'MISSION' 'CENTRAL'
 'TENDERLOIN' 'RICHMOND' 'SOUTHERN' 'PARK']


In [14]:
#drop address
test = test.drop('Address', axis=1)

In [15]:
test.head(1)

,Id,X,Y,date,time,Friday,Monday,Saturday,Sunday,Thursday,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,-122.399588,37.735051,1.0,86340,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
#not used
#get Category to front
tc = test.columns.tolist()
for i in range(0,4):
    tc.append(tc.pop(1))
    
# #add category column to front
# tc = [tc[0]] + sorted(categoryC) + tc[1:]

# #add blank category columns
# for i in categoryC:
#     test[i] = 0

#sort columns
test = test[tc]

### naive bayes test

In [17]:
train.head(1)

,Category,X,Y,date,time,Friday,Monday,Saturday,Sunday,Thursday,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,-122.425892,37.774599,1.0,85980,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [18]:
train_data = train.values

In [19]:
X = train_data[0::, 1::]

In [20]:
Y = train_data[0::, 0:1:]

In [65]:
print(X.shape, Y.shape)

(878049, 21) (878049, 1)


In [21]:
clf = GaussianNB()

In [22]:
clf.fit(X, Y.ravel())

GaussianNB()

In [64]:
test.head(1)

,Id,X,Y,date,time,Friday,Monday,Saturday,Sunday,Thursday,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,-122.399588,37.735051,1.0,86340,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
test_data = test.values[0::, 1::]

In [70]:
test_data.shape

(884262, 21)

In [25]:
p = clf.predict_proba(test_data)

In [37]:
subPTest = p[0:10]
print(subPTest.shape)

(39, 10)


In [39]:
subPTest[0]

array([ 0.03745814,  0.03755212,  0.03632981,  0.03473895,  0.03473895,
        0.03398467,  0.03489379,  0.03489795,  0.03934976,  0.03494361])

In [72]:
pLog = clf.predict_log_proba(test_data)

In [55]:
subT = np.around(p.T, 3)

In [73]:
subLogT = np.around(pLog.T, 3)

In [75]:
subLogT

array([[ -3.285,  -3.282,  -3.315, ...,  -3.404,  -3.326,  -3.424],
       [ -2.031,  -2.032,  -2.164, ...,  -1.911,  -1.872,  -1.959],
       [ -1.578,  -1.578,  -1.388, ...,  -2.323,  -2.352,  -2.258],
       ..., 
       [-11.632, -11.623, -12.15 , ..., -12.595, -11.723, -12.555],
       [ -5.687,  -5.679,  -6.016, ...,  -5.076,  -5.065,  -5.293],
       [-10.91 , -10.898, -10.678, ..., -10.878, -10.647, -10.39 ]])

In [76]:
with open("SFCrimeBayes.csv", "w") as f:
    o = csv.writer(f, delimiter=',')
    clist = categoryC
    o.writerow(["Id"] + clist)
    o.writerows(zip(test["Id"].values, *subLogT))

In [57]:
sub = pd.read_csv("./SFCrimeBayes.csv")

In [58]:
sub.head(1)

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.037,0.131,0.206,0.075,0.07,0.086,0.033,0.096,0.015,...,0.001,0.0,0.001,0.0,0.0,0.0,0.0,0.0,0.003,0.0


In [77]:
with open("./SFCrimeBayes.csv", 'rb') as f_in:
    with gzip.open("./SFCrimeBayes.gz", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

### SVM test

In [79]:
from sklearn import svm

In [80]:
clf = svm.SVC(decision_function_shape=’ovr’, probability=True)

In [ ]:
clf.fit(X, Y.ravel())

### train with random forest (잘안됨)

In [51]:
forest = RandomForestClassifier(n_estimators=75, verbose=1)

In [52]:
train.head(1)

,Category,X,Y,date,time,Friday,Monday,Saturday,Sunday,Thursday,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,-122.425892,37.774599,1.0,85980,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [53]:
test.head(1)

,Id,X,Y,date,time,Friday,Monday,Saturday,Sunday,Thursday,...,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,-122.399588,37.735051,1.0,86340,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
train_data = train.values

In [55]:
X = train_data[0::, 1::]

In [56]:
Y = train_data[0::, 0:1:].ravel()

In [57]:
forest = forest.fit(X, Y)

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:  3.3min
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  4.9min finished


In [58]:
print(train.shape, test.shape)

(878049, 22) (884262, 22)


In [59]:
#numpy array (remove id)
test_data = test.values[0::, 1::]

In [60]:
test_data.shape

(884262, 21)

In [61]:
output = forest.predict_proba(test_data)

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:  1.6min
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  2.4min finished


In [62]:
output.shape

(884262, 39)

In [63]:
forestT = np.around(output.T, 3)

In [64]:
with open("SFCrimeForest.csv", "w") as f:
    o = csv.writer(f, delimiter=',')
    clist = categoryC
    o.writerow(["Id"] + clist)
    o.writerows(zip(test["Id"].values, *forestT))

In [65]:
rf = pd.read_csv("./SFCrimeForest.csv")

In [66]:
rf.head(1)

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.031,0.136,0.244,0.04,0.107,0.116,0.213,0.027,0.024,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013,0.0


In [67]:
rf.shape

(884262, 40)

In [68]:
with open("./SFCrimeForest.csv", 'rb') as f_in:
    with gzip.open("./SFCrimeForest.gz", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)